# Parcel Logistics

## Table of Contents

* [1. Introduction](#introduction)
* [2. Mathematical Model](#model)
* [3. Data Source and Format](#data)
    * [3.1 Data Source](#data_source)
    * [3.2 Data Format](#data_format)
* [4. Conclusion](#conclustion)

<a name="introduction"></a>
### 1. Introduction ###

<a class="anchor" id="model"></a>
### 2. Mathematical Model ###

### 3. Data Source and Format<a class="anchor" id="data"></a>

#### 3.1 Data Source<a class="anchor" id="data_source"></a>

The data source we are using in this project is from a GitHub repository, https://github.com/austinlasseter/datasets-shipping-logistics

#### 3.2 Data Format<a class="anchor" id="data_format"></a>

<a id="conclusion"></a>
### 4. Conclusion ###

In [ ]:
# Install Ipopt
using Pkg
Pkg.add("Ipopt")
Pkg.add("PyPlot")

using JuMP, Ipopt
using PyPlot